In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

In [9]:
#load dataset
data = pd.read_csv('D:\Workspace\PhishCatchingModel\Dataset\dataset.csv')
#filter rows with missing values
data = data.dropna(axis=0)

data.head()

,id,NumDots,SubdomainLevel,PathLevel,UrlLength,NumDash,NumDashInHostname,AtSymbol,TildeSymbol,NumUnderscore,...,IframeOrFrame,MissingTitle,ImagesOnlyInForm,SubdomainLevelRT,UrlLengthRT,PctExtResourceUrlsRT,AbnormalExtFormActionR,ExtMetaScriptLinkRT,PctExtNullSelfRedirectHyperlinksRT,CLASS_LABEL
0,1,3,1,5,72,0,0,0,0,0,...,0,0,1,1,0,1,1,-1,1,1
1,2,3,1,3,144,0,0,0,0,2,...,0,0,0,1,-1,1,1,1,1,1
2,3,3,1,2,58,0,0,0,0,0,...,0,0,0,1,0,-1,1,-1,0,1
3,4,3,1,6,79,1,0,0,0,0,...,0,0,0,1,-1,1,1,1,-1,1
4,5,3,0,4,46,0,0,0,0,0,...,1,0,0,1,1,-1,0,-1,-1,1


In [10]:
data.columns

Index(['id', 'NumDots', 'SubdomainLevel', 'PathLevel', 'UrlLength', 'NumDash',
       'NumDashInHostname', 'AtSymbol', 'TildeSymbol', 'NumUnderscore',
       'NumPercent', 'NumQueryComponents', 'NumAmpersand', 'NumHash',
       'NumNumericChars', 'NoHttps', 'RandomString', 'IpAddress',
       'DomainInSubdomains', 'DomainInPaths', 'HttpsInHostname',
       'HostnameLength', 'PathLength', 'QueryLength', 'DoubleSlashInPath',
       'NumSensitiveWords', 'EmbeddedBrandName', 'PctExtHyperlinks',
       'PctExtResourceUrls', 'ExtFavicon', 'InsecureForms',
       'RelativeFormAction', 'ExtFormAction', 'AbnormalFormAction',
       'PctNullSelfRedirectHyperlinks', 'FrequentDomainNameMismatch',
       'FakeLinkInStatusBar', 'RightClickDisabled', 'PopUpWindow',
       'SubmitInfoToEmail', 'IframeOrFrame', 'MissingTitle',
       'ImagesOnlyInForm', 'SubdomainLevelRT', 'UrlLengthRT',
       'PctExtResourceUrlsRT', 'AbnormalExtFormActionR', 'ExtMetaScriptLinkRT',
       'PctExtNullSelfRedirectHyperl

In [11]:
#select features to work with
features = ['NumDots', 'SubdomainLevel', 'PathLevel', 'UrlLength', 'NumDash',
       'NumDashInHostname', 'AtSymbol', 'TildeSymbol', 'NumUnderscore',
       'NumPercent', 'NumQueryComponents', 'NumAmpersand', 'NumHash',
       'NumNumericChars', 'NoHttps', 'RandomString', 'IpAddress',
       'DomainInSubdomains', 'DomainInPaths', 'HttpsInHostname',
       'HostnameLength', 'PathLength', 'QueryLength', 'DoubleSlashInPath',
       'NumSensitiveWords', 'EmbeddedBrandName', 'PctExtHyperlinks',
       'PctExtResourceUrls', 'ExtFavicon', 'InsecureForms',
       'RelativeFormAction', 'ExtFormAction', 'AbnormalFormAction',
       'PctNullSelfRedirectHyperlinks', 'FrequentDomainNameMismatch',
       'FakeLinkInStatusBar', 'RightClickDisabled', 'PopUpWindow',
       'SubmitInfoToEmail', 'IframeOrFrame', 'MissingTitle',
       'ImagesOnlyInForm', 'SubdomainLevelRT', 'UrlLengthRT',
       'PctExtResourceUrlsRT', 'AbnormalExtFormActionR', 'ExtMetaScriptLinkRT',
       'PctExtNullSelfRedirectHyperlinksRT']

X = data[features]

#Choosing target label
y = data.CLASS_LABEL

In [13]:
#Shuffling the data
X, y = shuffle(X, y, random_state=42)

#splitting the data into train and test sets
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.2, random_state=42)

#Scaling the data
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
val_X = scaler.transform(val_X)

In [14]:
# NN Model
base_model_1 = keras.Sequential()

base_model_1.add(Dense(64, activation='relu', input_shape=(train_X.shape[1],)))
base_model_1.add(Dense(32, activation='relu'))
base_model_1.add(Dense(1, activation='sigmoid'))

base_model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

# Fit the NN Model
base_model_1.fit(train_X, train_y, epochs=50, batch_size=64, validation_data=(val_X, val_y), callbacks=[early_stopping, model_checkpoint])

Epoch 1/50
125/125 [==============================] - 1s 3ms/step - loss: 0.2817 - accuracy: 0.9032 - val_loss: 0.1722 - val_accuracy: 0.9410

Epoch 00001: val_loss improved from inf to 0.17218, saving model to best_model.h5
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.1385 - accuracy: 0.9484 - val_loss: 0.1486 - val_accuracy: 0.9505

Epoch 00002: val_loss improved from 0.17218 to 0.14860, saving model to best_model.h5
Epoch 3/50
125/125 [==============================] - 0s 2ms/step - loss: 0.1162 - accuracy: 0.9563 - val_loss: 0.1340 - val_accuracy: 0.9530

Epoch 00003: val_loss improved from 0.14860 to 0.13404, saving model to best_model.h5
Epoch 4/50
125/125 [==============================] - 0s 2ms/step - loss: 0.1014 - accuracy: 0.9606 - val_loss: 0.1279 - val_accuracy: 0.9525

Epoch 00004: val_loss improved from 0.13404 to 0.12792, saving model to best_model.h5
Epoch 5/50
125/125 [==============================] - 0s 2ms/step - loss: 0.0905 - accur

In [15]:
#Random Forest Model
base_model_2 = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
# Fit the Random Forest Model
base_model_2.fit(train_X, train_y)

#Gradient Boosting Model
base_model_3 = GradientBoostingClassifier(n_estimators=70, learning_rate=0.1, max_depth=5, random_state=12)
# Fit the Gradient Boosting Model
base_model_3.fit(train_X, train_y)

#SVC Model
base_model_4 = SVC(probability=True, random_state=10, C=1.0, kernel='rbf')  # SVC with probability=True
# Fit the SVC Model
base_model_4.fit(train_X, train_y)

#Logistic Regression
base_model_5 = LogisticRegression(random_state=52, C=6, max_iter=500)
# Fit the Logistic Regression Model
base_model_5.fit(train_X, train_y)


LogisticRegression(C=6, max_iter=500, random_state=52)

In [18]:
# Define the models
models = {
    "NN Model": base_model_1,
    "Random Forest": base_model_2,
    "Gradient Boosting": base_model_3,
    "SVC": base_model_4,
    "Logistic Regression": base_model_5
}

# Evaluate each model
for name, model in models.items():
    if name == "NN Model":
        pred = model.predict(val_X).flatten()  # NN model returns probabilities
    else:
        pred = model.predict(val_X)  # Other models return direct predictions
    
    # Convert predictions to binary
    binary_predictions = (pred > 0.5).astype(int)
    
    # Calculate accuracy
    accuracy = accuracy_score(val_y, binary_predictions)
    print(f"{name} Accuracy: {accuracy * 100:.2f}%")

NN Model Accuracy: 96.85%
Random Forest Accuracy: 97.40%
Gradient Boosting Accuracy: 98.00%
SVC Accuracy: 95.55%
Logistic Regression Accuracy: 94.50%


In [20]:
from sklearn.ensemble import VotingClassifier

# Define the models for the ensemble (excluding the NN model)
ensemble_models = [
    ('rf', base_model_2),
    ('gb', base_model_3),
    ('svc', base_model_4),
    ('lr', base_model_5)
]

# Create a VotingClassifier with weights
ensemble_model = VotingClassifier(estimators=ensemble_models, voting='soft', weights=[0.3, 0.3, 0.2, 0.2])

# Fit the ensemble model on your training data
ensemble_model.fit(train_X, train_y)

# Make predictions using the ensemble model
ensemble_predictions = ensemble_model.predict(val_X)

# Calculate the accuracy of the ensemble model
ensemble_accuracy = accuracy_score(val_y, ensemble_predictions)
print(f'Ensemble Accuracy: {ensemble_accuracy * 100:.2f}%')


Ensemble Accuracy: 97.80%


In [21]:
# Classification report
classification_rep = classification_report(val_y, ensemble_predictions)
print("Classification Report:\n", classification_rep)

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.98      0.98      1008
           1       0.98      0.97      0.98       992

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000



In [22]:
import joblib

# Save the trained model to a file
joblib.dump(ensemble_model, 'ensemble_model.h5')

['ensemble_model.h5']